In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import timedelta, date, datetime
from config import api_key
from pprint import pprint

In [2]:
base_url = "https://api.nytimes.com/svc/books/v3"
tail = f"?api-key={api_key}"

In [6]:
#since we are expanding our scope i need a date calculator to generate a list of weeks
weeks_in_bc=[] 
counter = 0
week = timedelta(days=7)
start=datetime.strptime('2018 12 31', '%Y %m %d').date()

In [7]:
while counter < 91:
    pub_date = (start+week)
    pubdateform = pub_date.isoformat()
    weeks_in_bc.append(pubdateform)
    start = pub_date
    if counter > 93:
        break
    counter += 1

In [8]:
print(weeks_in_bc)

['2019-01-07', '2019-01-14', '2019-01-21', '2019-01-28', '2019-02-04', '2019-02-11', '2019-02-18', '2019-02-25', '2019-03-04', '2019-03-11', '2019-03-18', '2019-03-25', '2019-04-01', '2019-04-08', '2019-04-15', '2019-04-22', '2019-04-29', '2019-05-06', '2019-05-13', '2019-05-20', '2019-05-27', '2019-06-03', '2019-06-10', '2019-06-17', '2019-06-24', '2019-07-01', '2019-07-08', '2019-07-15', '2019-07-22', '2019-07-29', '2019-08-05', '2019-08-12', '2019-08-19', '2019-08-26', '2019-09-02', '2019-09-09', '2019-09-16', '2019-09-23', '2019-09-30', '2019-10-07', '2019-10-14', '2019-10-21', '2019-10-28', '2019-11-04', '2019-11-11', '2019-11-18', '2019-11-25', '2019-12-02', '2019-12-09', '2019-12-16', '2019-12-23', '2019-12-30', '2020-01-06', '2020-01-13', '2020-01-20', '2020-01-27', '2020-02-03', '2020-02-10', '2020-02-17', '2020-02-24', '2020-03-02', '2020-03-09', '2020-03-16', '2020-03-23', '2020-03-30', '2020-04-06', '2020-04-13', '2020-04-20', '2020-04-27', '2020-05-04', '2020-05-11', '2020

In [5]:
#Variables for lists/API call
result_count=np.arange(15)
list_name=[]
publish_date=[]
title_name=[]
author_name=[]
weeks_on_list=[]
#not used in df still figuring out
isbns=[]

In [6]:
for week in weeks_in_bc:
    response = requests.get(f"{base_url}/lists/{week}/hardcover-fiction.json{tail}")
    data=response.json()
    for i in result_count:
        listname = data['results']['list_name']
        list_name.append(listname)
        pub_date = data['results']['bestsellers_date']
        publish_date.append(pub_date)
        title = data['results']['books'][i]['title']
        title_name.append(title)
        author = data['results']['books'][i]['author']
        author_name.append(author)
        weeks = data['results']['books'][i]['weeks_on_list']
        weeks_on_list.append(weeks)
        isbnlist = data['results']['books'][i]['isbns']
        isbns.append(isbnlist)

In [20]:
pprint(data)

{'copyright': 'Copyright (c) 2020 The New York Times Company.  All Rights '
              'Reserved.',
 'last_modified': '2020-06-24T23:39:01-04:00',
 'num_results': 15,
 'results': {'bestsellers_date': '2020-06-20',
             'books': [{'age_group': '',
                        'amazon_product_url': 'https://www.amazon.com/dp/0316420042?tag=NYTBSREV-20&tag=NYTBS-20',
                        'article_chapter_link': '',
                        'asterisk': 0,
                        'author': 'Elin Hilderbrand',
                        'book_image': 'https://s1.nyt.com/du/books/images/9780316420044.jpg',
                        'book_image_height': 500,
                        'book_image_width': 322,
                        'book_review_link': '',
                        'book_uri': 'nyt://book/10bc4fb8-3279-533a-be20-ef6dd6cfb1be',
                        'buy_links': [{'name': 'Amazon',
                                       'url': 'https://www.amazon.com/dp/0316420042?tag=NYTBSREV-

In [9]:
hc_fiction_df=pd.DataFrame({
    "List": list_name,
    "List_Date": publish_date,
    "Title": title_name,
    "Author": author_name,
    "Weeks_On_List": weeks_on_list 
})

In [10]:
hc_fiction_df.head()

,List,List_Date,Title,Author,Weeks_On_List
0,Hardcover Fiction,2020-09-19,THE EVENING AND THE MORNING,Ken Follett,1
1,Hardcover Fiction,2020-09-19,TROUBLED BLOOD,Robert Galbraith,1
2,Hardcover Fiction,2020-09-19,VINCE FLYNN: TOTAL POWER,Kyle Mills,1
3,Hardcover Fiction,2020-09-19,TO SLEEP IN A SEA OF STARS,Christopher Paolini,1
4,Hardcover Fiction,2020-09-19,THE VANISHING HALF,Brit Bennett,16


In [11]:
hc_fiction_df["Title"].nunique()

65

In [12]:
hc_titles = list(hc_fiction_df["Title"].unique())

In [13]:
#Some pbj: I need to make a new df with only the entries for the 65 titles with thier max weeks on the list...
#trying groupby title pls hold... this did not work
#trying sort and drop dupes... THIS WORKED

In [14]:
test_df = hc_fiction_df.copy().sort_values(by=["Title","Weeks_On_List"], ascending=False)

In [15]:
only_hc_titles_df = test_df.copy().drop_duplicates(subset="Title", keep="first", inplace=False)

In [16]:
only_hc_titles_df = only_hc_titles_df.sort_values(by="Weeks_On_List", ascending=False)

In [17]:
only_hc_titles_df.head()

,List,List_Date,Title,Author,Weeks_On_List
5,Hardcover Fiction,2020-09-19,WHERE THE CRAWDADS SING,Delia Owens,107
55,Hardcover Fiction,2020-08-29,THE SILENT PATIENT,Alex Michaelides,56
27,Hardcover Fiction,2020-09-12,AMERICAN DIRT,Jeanine Cummins,34
59,Hardcover Fiction,2020-08-29,IF IT BLEEDS,Stephen King,18
72,Hardcover Fiction,2020-08-22,CAMINO WINDS,John Grisham,17


In [18]:
counts = pd.DataFrame(hc_fiction_df["Title"].value_counts())
counts

,Title
WHERE THE CRAWDADS SING,14
THE VANISHING HALF,14
THE GUEST LIST,14
AMERICAN DIRT,13
28 SUMMERS,11
...,...
CHOPPY WATER,1
THE DARKEST EVENING,1
FRIENDS AND STRANGERS,1
DARK SONG,1


In [19]:
#have to take a break, daily limit reached. oops